# Preparing Building Electricity consumption data
---

### Overview
1. Inspect raw data
2. Process building imagery data
3. Process meteo and load profile data
---

We start with importing hyper parameters and modules

In [1]:
import hyper_buildingelectricity
import prep_building_electricity
import pandas as pd

HYPER = hyper_buildingelectricity.HyperBuildingElectricity()

### 1. Inspect raw data

In [ ]:
df_consumption, df_building_images, df_meteo_dict = prep_building_electricity.import_all_data(HYPER)

display(df_consumption)
display(df_building_images)
display(df_meteo_dict['meteo_1_2014.csv'])

### 2. Process building imagery data

In [ ]:
prep_building_electricity.process_building_imagery(HYPER, df_building_images)

### 3. Process meteo and load profile data

In [ ]:
prep_building_electricity.process_meteo_and_load_profiles(HYPER, df_consumption, df_meteo_dict)